In [1]:
import sqlite3
import pandas as pd
con = sqlite3.connect("C:/Users/ams-duongn.EXPEDITORS/Desktop/learn/Data Science Master/M3W1/chinook.db")

In [2]:
# I would like to show the list of albums and tracks per artist
artist_album_track = pd.read_sql_query('SELECT r.Name as Artist_Name,\
                            l.Title as Album_Title, t.Name as Track_Name\
                            FROM albums as l\
                            INNER JOIN artists as r on l.ArtistId = r.ArtistId\
                            INNER JOIN tracks as t on l.AlbumId = t.AlbumId', con)
artist_album_track

,Artist_Name,Album_Title,Track_Name
0,AC/DC,For Those About To Rock We Salute You,For Those About To Rock (We Salute You)
1,AC/DC,For Those About To Rock We Salute You,Put The Finger On You
2,AC/DC,For Those About To Rock We Salute You,Let's Get It Up
3,AC/DC,For Those About To Rock We Salute You,Inject The Venom
4,AC/DC,For Those About To Rock We Salute You,Snowballed
...,...,...,...
3498,Eugene Ormandy,Respighi:Pines of Rome,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...
3499,Emerson String Quartet,Schubert: The Late String Quartets & String Qu...,"String Quartet No. 12 in C Minor, D. 703 ""Quar..."
3500,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Monteverdi: L'Orfeo,"L'orfeo, Act 3, Sinfonia (Orchestra)"
3501,Nash Ensemble,Mozart: Chamber Music,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


In [3]:
# I would like to extract the Artists and Albums of the Top 10 Tracks by Sale Amount
top_10_sale = pd.read_sql_query('WITH TOP_10_TRACK AS (\
                                 SELECT B.TrackId, B.Sale\
                                 FROM (\
                                 SELECT A.TrackId, A.Sale, ROW_NUMBER() OVER (ORDER BY A.Sale DESC) AS SEQ\
                                 FROM (\
                                 SELECT ii.TrackId, Sum(ii.UnitPrice * ii.Quantity) as Sale\
                                 FROM invoice_items as ii group by ii.TrackId) A\
                                 ) B WHERE B.SEQ <= 10)\
                                 SELECT r.Name as Artist_Name, l.Title as Album_Title,\
                                        t.Name as Track_Name, TOP_10_TRACK.Sale as Track_Sale\
                                 FROM TOP_10_TRACK\
                                 INNER JOIN tracks as t on t.TrackId = TOP_10_TRACK.TrackId\
                                 INNER JOIN albums as l on l.AlbumId = t.AlbumId\
                                 INNER JOIN artists as r on r.ArtistId = l.ArtistId', con)
top_10_sale

,Artist_Name,Album_Title,Track_Name,Track_Sale
0,Battlestar Galactica,"Battlestar Galactica, Season 3",The Woman King,3.98
1,Heroes,"Heroes, Season 1",The Fix,3.98
2,Lost,"Lost, Season 1",Walkabout,3.98
3,The Office,"The Office, Season 1",Hot Girl,3.98
4,The Office,"The Office, Season 3",Gay Witch Hunt,3.98
5,The Office,"The Office, Season 3",Phyllis's Wedding,3.98
6,Heroes,"Heroes, Season 1",How to Stop an Exploding Man,3.98
7,Aquaman,Aquaman,Pilot,3.98
8,Battlestar Galactica,"Battlestar Galactica, Season 3",Occupation / Precipice,1.99
9,Battlestar Galactica,"Battlestar Galactica, Season 3","Exodus, Pt. 1",1.99


In [4]:
sales_day_2012 = pd.read_sql_query('SELECT i.InvoiceDate,i.BillingAddress, i.BillingCity,\
                                    (C.FirstName || " " || c.LastName) as Customer_Name,\
                                    Sum(ii.UnitPrice * ii.Quantity) as Sale\
                                    FROM invoices i\
                                    INNER JOIN invoice_items ii on ii.InvoiceId = i.InvoiceId\
                                    INNER JOIN customers c on c.CustomerId = i.CustomerId\
                                    WHERE date(i.InvoiceDate) between date("2012-01-01") and date("2012-12-31")\
                                    GROUP BY i.InvoiceDate,i.BillingAddress, i.BillingCity', con)
sales_day_2012

,InvoiceDate,BillingAddress,BillingCity,Customer_Name,Sale
0,2012-01-01 00:00:00,421 Bourke Street,Sidney,Mark Taylor,13.86
1,2012-01-09 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,Eduardo Martins,0.99
2,2012-01-22 00:00:00,"Av. Paulista, 2022",São Paulo,Alexandre Rocha,1.98
3,2012-01-22 00:00:00,Qe 7 Bloco G,Brasília,Fernanda Ramos,1.98
4,2012-01-23 00:00:00,700 W Pender Street,Vancouver,Jennifer Peterson,3.96
...,...,...,...,...,...
78,2012-12-15 00:00:00,700 W Pender Street,Vancouver,Jennifer Peterson,0.99
79,2012-12-28 00:00:00,1600 Amphitheatre Parkway,Mountain View,Frank Harris,1.98
80,2012-12-28 00:00:00,627 Broadway,New York,Michelle Brooks,1.98
81,2012-12-29 00:00:00,541 Del Medio Avenue,Mountain View,Dan Miller,3.96
